In [21]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
nltk.download("stopwords")
nltk.download("punkt_tab")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\imane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\imane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [23]:
stop_words = set(stopwords.words("english"))
#lemmatizer = WordNetLemmatizer()

# Function to clean and preprocess the text
def preprocess_text(text):
    # Remove non-alphabetic characters and lower case the text
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    #tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return ' '.join(tokens)



In [24]:
# Load the datasets (replace paths with your actual file paths)
resumes = pd.read_csv("resumes.csv")  # Columns: ["Category", "Resume"]
jobs = pd.read_csv("jobs.csv")  # Columns: ["Job Title", "Company", "Description", "Language"]


# Apply preprocessing to job descriptions and resumes
jobs['processed_description'] = jobs['Description'].apply(preprocess_text)
jobs = jobs[jobs["Language"] == "en"]
resumes['processed_text'] = resumes['Resume'].apply(preprocess_text)

In [25]:
# Combine both job and resume texts into one list of sentences
combined_corpus = jobs['processed_description'].tolist() + resumes['processed_text'].tolist()

def train_word2vec_model(corpus):
    model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)
    return model

word2vec_model = train_word2vec_model(combined_corpus)


In [29]:
# Example: Check if the word 'python' is in the vocabulary
query_word = 'python'.lower()  # Normalize the word to lowercase to match preprocessing
if query_word in word2vec_model.wv:
    # Get the word embedding
    word_embedding = word2vec_model.wv[query_word]
    print(f"Embedding for '{query_word}':\n{word_embedding}")
else:
    print(f"Word '{query_word}' not found in the vocabulary.")

# Example: Get the top 5 most similar words to 'python'
if query_word in word2vec_model.wv:
    similar_words = word2vec_model.wv.most_similar(query_word, topn=5)
    print(f"Top 5 words most similar to '{query_word}':")
    for word, similarity in similar_words:
        print(f"{word}: {similarity}")
else:
    print(f"Word '{query_word}' not found in the vocabulary.")

Word 'python' not found in the vocabulary.
Word 'python' not found in the vocabulary.


In [ ]:

# Function to get the document embedding by averaging word embeddings
def get_document_embedding(text_tokens, model):
    embeddings = []
    for word in text_tokens:
        if word in model.wv:
            embeddings.append(model.wv[word])
    
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)  # If no valid embeddings, return zero vector
    
    return np.mean(embeddings, axis=0)

# Get document embeddings for jobs and resumes
job_embeddings = jobs['processed_description'].apply(lambda x: get_document_embedding(x, word2vec_model))
resume_embeddings = resumes['processed_text'].apply(lambda x: get_document_embedding(x, word2vec_model))

# Compute cosine similarity between each resume and job
similarity_scores = []
for resume_embedding in resume_embeddings:
    job_similarities = cosine_similarity([resume_embedding], job_embeddings.tolist())[0]
    similarity_scores.append(job_similarities)


In [ ]:
# Number of top matches you want to keep
top_n_matches = 10

# Store matches for each resume
matches = []
for i, resume in resumes.iterrows():
    # Get the top N job indices based on similarity scores
    top_job_indices = np.argsort(similarity_scores[i])[-top_n_matches:][::-1]
    
    # Get the matched jobs and their similarities
    top_jobs = jobs.iloc[top_job_indices]
    top_similarities = [similarity_scores[i][j] for j in top_job_indices]
    
    matches.append({
        "Resume_ID": resume.name,
        "Top_Matched_Jobs": top_jobs['Job Title'].tolist(),
        "Similarities": top_similarities
    })

# Example output for the first resume
print(matches[0])
